In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import os
# Seu trabalho será armazenado em uma pasta chamada `minicurso_rl` por padrão 
# para evitar que o tempo limite da instância do Colab exclua suas edições

DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/minicurso_rl"
DRIVE_PYTHON_PATH = DRIVE_PATH.replace("\\", "")
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

SYM_PATH = "/content/drive/MyDrive/Colab Notebooks/minicurso_rl"
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [5]:
!pip install -U cloudpickle > /dev/null 2>&1 
!pip install "gym[all]" > /dev/null 2>&1 
!pip install "gym[box2d]" > /dev/null 2>&1 
!pip install "stable-baselines3[extra]" > /dev/null 2>&1 

!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

!pip install pyvirtualdisplay > /dev/null 2>&1

!pip install plotly > /dev/null 2>&1

In [6]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/ -y
! python -m atari_py.import_roms /content/ROM/ > /dev/null 2>&1

--2021-11-20 02:24:23--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  10.61M   625KB/s    in 18s     

2021-11-20 02:24:41 (603 KB/s) - ‘Roms.rar’ saved [11128004/11128004]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar

Extracting  /content/ROM/HC ROMS.zip                                      36%  OK 
Extracting  /content/ROM/ROMS.zip                                         74% 99%  OK 
All OK


In [7]:
import torch
import random
import numpy as np

torch.manual_seed(10)
random.seed(10)
np.random.seed(10)

In [8]:
# Procedimento para renderizar o ambiente no Google Colab

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()


from matplotlib import pyplot as plt, animation
%matplotlib inline
from IPython import display

def create_anim(frames, dpi, fps):
    plt.figure(figsize=(frames[0].shape[1] / dpi, frames[0].shape[0] / dpi), dpi=dpi)
    patch = plt.imshow(frames[0])
    def setup():
        plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, init_func=setup, frames=len(frames), interval=fps)
    return anim

def display_anim(frames, dpi=72, fps=60):
    anim = create_anim(frames, dpi, fps)
    return anim.to_jshtml()

def save_anim(frames, filename, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    anim.save(filename)


class trigger:
    def __init__(self):
        self._trigger = True

    def __call__(self, e):
        return self._trigger

    def set(self, t):
        self._trigger = t

In [9]:
import gym
environment_id = "EnduroNoFrameskip-v4"       # Nome do ambiente utilizado

In [10]:
env = gym.make(environment_id)                # Criando o ambiente

frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     # Retorna a observação inicial
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))     # Renderizando o ambiente
        action = env.action_space.sample()              # Seleciona uma ação aleatória
        obs, reward, done, info = env.step(action)    # Executa a ação selecionada
        score += reward
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()



Episódio: 1 Pontuação: 0.0


In [11]:
display.HTML(display_anim(frames))

Output hidden; open in https://colab.research.google.com to view.

In [12]:
!cd minicurso_rl && gdown --id 1ZV5fvCbU_gbTy1AraSR-ymNsiNQesBvt

/bin/bash: line 0: cd: minicurso_rl: No such file or directory


In [13]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.atari_wrappers import AtariWrapper
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack

In [17]:
expert = PPO.load("EnduroNoFrameskip-v4")

In [18]:
env = make_vec_env(environment_id, wrapper_class=AtariWrapper)  
env = VecFrameStack(env, 4)

In [19]:
mean_reward, std_reward = evaluate_policy(expert, env, n_eval_episodes=10)
print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 1164.8 +/- 175.64726015511886


In [20]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     # Retorna a observação inicial
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))         # Renderizando o ambiente
        action = expert.predict(obs, deterministic=True)    # Seleciona uma ação do agente especialista
        obs, reward, done, info = env.step(action)          # Executa a ação selecionada
        score += reward
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()

display.HTML(display_anim(frames))

Output hidden; open in https://colab.research.google.com to view.

In [21]:
from tqdm import tqdm

In [22]:
num_interactions = int(4e4)

In [27]:
if isinstance(env.action_space, gym.spaces.Box):
    expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
    expert_actions = np.empty((num_interactions,) + (env.action_space.shape[0],))
else:
    expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
    expert_actions = np.empty((num_interactions,) + env.action_space.shape)

# HW: Interaja  com o ambiente `env` conforme visto anteriormente. Armazene 
# as observações e as ações do especialista em `expert_observations` e 
# `expert_actions` respectivamente para construir o dataset.

#frames = []
episodes = 1
i = 0
while num_interactions > i:
#for episode in range(1, episodes+1):
    obs = env.reset()                                       # Retorna a observação inicial
    done = False
    score = 0
    while not done and num_interactions > i:
        #frames.append(env.render(mode='rgb_array'))        # Renderizando o ambiente
        action = expert.predict(obs, deterministic=True)    # Seleciona uma ação do agente especialista
        expert_observations[i] = obs
        expert_actions[i] = action[0][0]
        obs, reward, done, info = env.step(action)          # Executa a ação selecionada
        score += reward
        i+=1
    print("\n\nEpisódio: {} Pontuação: {}".format(episodes,score))
    episodes += 1
env.close()

# Salva os dados (observação, ação)
np.savez_compressed(
    "/content/drive/MyDrive/Colab Notebooks/minicurso_rl/expert_data",
    expert_actions=expert_actions,
    expert_observations=expert_observations,
)



Episódio: 1 Pontuação: [609.]


Episódio: 2 Pontuação: [1084.]


Episódio: 3 Pontuação: [1381.]


Episódio: 4 Pontuação: [177.]


In [28]:
try:
    expert_observations, expert_actions
except NameError:
    pass
else:
  del expert_observations, expert_actions

# Carrega os dados salvos
data = np.load("/content/drive/MyDrive/Colab Notebooks/minicurso_rl/expert_data.npz")

In [29]:
try:
    expert_observations, expert_actions
except NameError:
    pass
else:
  del expert_observations, expert_actions


# Carrega os dados salvos
data = np.load("/content/drive/MyDrive/Colab Notebooks/minicurso_rl/expert_data.npz")

In [30]:
data

Para usar perfeitamente o PyTorch no processo de treinamento, criamos uma subclasse de ExpertDataset do Dataset base do Pytorch
Observe que inicializamos o conjunto de dados com as observações e ações de especialistas geradas anteriormente.
Implementamos ainda as funções mágicas __getitem__ e__len__ do Python para permitir que o manuseio do conjunto de dados do PyTorch acesse linhas arbitrárias no conjunto de dados e informá-lo sobre o comprimento do conjunto de dados.
Para obter mais informações sobre os conjuntos de dados de PyTorch, você pode ler: https://pytorch.org/docs/stable/data.html.

In [31]:
from torch.utils.data.dataset import Dataset, random_split

In [32]:
class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions
        
    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

Agora instanciamos o ExpertDataSet e o dividimos em conjuntos de dados de treinamento e teste.

In [33]:
expert_dataset = ExpertDataSet(data["expert_observations"], data["expert_actions"])

del data

train_size = int(0.8 * len(expert_dataset))     # 80% dos dados para treinamento
test_size = len(expert_dataset) - train_size    # E o restante dos dados para teste

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

In [34]:
print("# test_expert_dataset: ", len(test_expert_dataset))
print("# train_expert_dataset: ", len(train_expert_dataset))

# test_expert_dataset:  8000
# train_expert_dataset:  32000


# Treinar o agente estudante
## Nossos próximos passos:

Extraímos a rede de políticas de nosso aluno.
Carregamos o conjunto de dados de especialistas (rotulados) contendo observações de especialistas como entradas e ações de especialistas como alvos.
Realizamos aprendizagem supervisionada, ou seja, ajustamos os parâmetros da rede de políticas de forma que, dadas as observações de especialistas como entradas para a rede, suas saídas correspondam aos alvos (ações de especialistas).
Ao treinar a rede de políticas dessa maneira, o agente aluno correspondente é ensinado a se comportar como o agente especialista que foi usado para criar o conjunto de dados especialista (Behavior Cloning).

In [35]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.optim.lr_scheduler import StepLR

In [36]:
batch_size=128
epochs=20
scheduler_gamma=0.99
learning_rate=5e-3
log_interval=100
no_cuda=False    
seed=1
test_batch_size=128

In [37]:
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

Agora iremos definir a rede neural que iremos utilizar para o aluno estudante. Aqui criamos um agente de aprendizado por reforço, e extraimos dele a rede da política. Alternativamente você pode construir a sua própria rede neural.

Como estamos utilizando imagens como entrada, iremos utilizar uma rede chamada de Rede Neural Convolucional (Convolutional Neural Network - CNN).


*  https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-cnn-26a14c2ea29

*  https://medium.com/swlh/introduction-to-cnn-image-classification-using-cnn-in-pytorch-11eefae6d83c

*  https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/


In [38]:
from torchsummary import summary

student = PPO('CnnPolicy', env, verbose=1)

# Extrair politica inicial
model = student.policy.to(device)

# Mostra um sumário da rede, mostrando todas as suas camadas 
summary(model, (4, 84, 84))

Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 20, 20]           8,224
              ReLU-2           [-1, 32, 20, 20]               0
            Conv2d-3             [-1, 64, 9, 9]          32,832
              ReLU-4             [-1, 64, 9, 9]               0
            Conv2d-5             [-1, 64, 7, 7]          36,928
              ReLU-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
            Linear-8                  [-1, 512]       1,606,144
              ReLU-9                  [-1, 512]               0
        NatureCNN-10                  [-1, 512]               0
     MlpExtractor-11     [[-1, 512], [-1, 512]]               0
           Linear-12                    [-1, 1]             513
           Linear-13                    [-1, 

In [39]:
nb_actions = env.action_space.n
print("O número total de ações possíveis é: ", nb_actions)

O número total de ações possíveis é:  9


In [40]:
# HW: Implementar função de Loss
criterion = nn.CrossEntropyLoss()

In [41]:
# HW: Implementar função de Acurácia
def acc(model_out, true_out):
    model_a = model_out.argmax(1)
    true_a = true_out
    return torch.sum(model_a == true_a)

In [42]:
# HW: Implementar função de Treino
# ela deve retornar informações de loss e acurácia
def train():
    _loss = 0.0
    _acc = 0.0

    model.train()
    for data, target in train_loader:
        data = data.permute(0, 3, 1, 2)
        data, target = data.to(device), target.to(device)

        if isinstance(env.action_space, gym.spaces.Box):
            # A2C/PPO policy outputs actions, values, log_prob
            action, _, _ = model(data)
            action_prediction = action.double()
        else:
            # Retrieve the logits for A2C/PPO when using discrete actions
            latent_pi, _, _ = model._get_latent(data)
            logits = model.action_net(latent_pi)
            action_prediction = logits
            target = target.long()

        train_loss = criterion(action_prediction, target)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        _loss += train_loss.data.cpu().numpy()
        _acc += acc(action_prediction, target).data.cpu().numpy()
    
    _loss /= float(len(train_loader.dataset))
    _acc /= float(len(train_loader.dataset))
    print(f"Train Set: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")
    return _loss, _acc

In [50]:
def test():
    _loss = 0.0
    _acc = 0.0

    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data = data.permute(0, 3, 1, 2)
            data, target = data.to(device), target.to(device)

            if isinstance(env.action_space, gym.spaces.Box):
                # A2C/PPO policy outputs actions, values, log_prob
                action, _, _ = model(data)
                action_prediction = action.double()
            else:
                # Retrieve the logits for A2C/PPO when using discrete actions
                latent_pi, _, _ = model._get_latent(data)
                logits = model.action_net(latent_pi)
                action_prediction = logits
                target = target.long()
            
            test_loss = criterion(action_prediction, target)
            _loss += test_loss.data.cpu().numpy()
            _acc += acc(action_prediction, target).data.cpu().numpy()

    _loss /= float(len(test_loader.dataset))
    _acc /= float(len(test_loader.dataset))
    print(f"Test Set: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")
    return _loss, _acc

In [51]:
mean_reward, std_reward = evaluate_policy(student, env, n_eval_episodes=10)
print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 0.0 +/- 0.0


In [52]:
# Aqui, usamos PyTorch `DataLoader` para carregar o `ExpertDataset` criado anteriormente para treinamento e teste
train_loader = torch.utils.data.DataLoader(
    dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
)

# Defina um Otimizador e uma programação de taxa de aprendizagem (learning rate).
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

In [49]:
# Agora estamos finalmente prontos para treinar o modelo de política.
train_loss, train_acc = [], []
test_loss, test_acc = [], []
_learning_rate = []

for epoch in range(1, epochs + 1):
    _learning_rate.append(scheduler.get_lr()[0])
    print("learning rate: ", scheduler.get_lr()[0])

    _train_loss, _train_acc = train()
    _test_loss, _test_acc = test()

    train_loss.append(_train_loss)
    train_acc.append(_train_acc)
    test_loss.append(_test_loss)
    test_acc.append(_test_acc)
    
    scheduler.step()

learning rate:  0.005


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


AttributeError: ignored

In [ ]:
import plotly.graph_objs as go

In [ ]:
fig = go.Figure([
    go.Scatter(
        y=train_acc,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Acurácia de Treino"
    ),
    go.Scatter(
        y=test_acc,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Acurácia de Teste"
    ),
])
fig.update_layout(
    title="Acurácia",
    yaxis = dict(
        tickformat = "%",
    ),
    xaxis = dict(
        title = "Época",
    )
)
fig.show()

In [ ]:
fig = go.Figure([
    go.Scatter(
        y=train_loss,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Loss de Treinamento"
    ),
    go.Scatter(
        y=test_loss,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Loss de Teste"
    ),
])
fig.update_layout(
    title="Loss",
    xaxis = dict(title="Época")
)
fig.show()

In [ ]:
fig = go.Figure([
    go.Scatter(
        y=_learning_rate,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Learning Rate"
    ),
])
fig.update_layout(
    title="Learning Rate",
    xaxis = dict(title="Época")
)
fig.show()

In [ ]:
# Inserir a rede treinada de volta no agente estudante
student.policy = model

In [ ]:
mean_reward, std_reward = evaluate_policy(student, env, n_eval_episodes=10)
print(f"Mean reward = {mean_reward} +/- {std_reward}")

In [ ]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))           
        action = student.predict(obs, deterministic=True)   
        obs, reward, done, info = env.step(action)       
        score += reward
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()

In [ ]:
display.HTML(display_anim(frames))